<a href="https://colab.research.google.com/github/sakibchowdhury131/Emotion_detection/blob/master/sswe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu May  7 22:49:58 2020

@author: sakib
"""


import os
import pandas as pd
import numpy as np
from zipfile import ZipFile
import urllib.request
from tempfile import mktemp

base_path=r'/content'
base_folder='data'

# URL to download the sentiment140 dataset
data_url='http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip'

def change_base_dir(base_dir_path):
    """ Change the working directopry of the code"""
    
    if not os.path.exists(base_dir_path):
        print ('creating directory', base_dir_path)
        os.makedirs(base_dir_path)
    print ('Changing base directory to ', base_dir_path)
    os.chdir(base_dir_path)

def download_data(download_url, filename='downloaded_data.zip'):
    """ Download and extract data """
    
    downloaded_filename = os.path.join('.', filename)
    print ('Step 1: Downloading data')
    urllib.request.urlretrieve(download_url,downloaded_filename)
    print ('Step 2: Extracting data')
    zipfile=ZipFile(downloaded_filename)
    zipfile.extractall('./')
    zipfile.close()

def extract_tweets_and_labels(filename ):
    """ Extract tweets and labels from the downloaded data"""
    
    print ('Step 3: Reading the data as a dataframe')
    df=pd.read_csv(filename, header=None, encoding='iso-8859-1')    
    df.columns=['Label','TweetId','Date','Query','User','Text']
    print ('Read {} lines'.format(df.shape[0]))
    print ('Discarding neutral tweets')
    df=df[df.Label!=2]
    print ('No of lines in the data after filtering neutral tweets: {}'.format(df.shape[0]))
    print ('Step 4: Shuffling the data')
    train_length=int(df.shape[0]*0.8)    
    df=df.sample(frac=1) # reshuffling the data
      
    df['Text']=df['Text'].astype(str).apply(lambda x:x.strip())#.encode('ascii','ignore')#str.decode('utf8','ignore')#.str.encode('ascii','ignore')
    print (df.head())
    print ('Step 5: Dividing into test and train datasets')
    df_train = df.iloc[:train_length, :]
    df_test = df.iloc[train_length:, :]    
    
    print ('Step 6: Exporting the train and test datasets')    
    print ('Exporting training data of rows {}'.format(df_train.shape[0]))
    export_prefix='training'
    df_train[['Label']].to_csv(export_prefix+'_label.csv', header=False, index=False)
    df_train[['Text']].to_csv(export_prefix+'_text.csv', header=False, index=False)
    print ('Target distribution in the training data is as follows')
    print ('\n',df_train['Label'].value_counts()) 
    
    print ('Exporting training data of rows {}'.format(df_test.shape[0]))
    export_prefix='testing'
    df_test[['Label']].to_csv(export_prefix+'_label.csv', header=False, index=False)
    df_test[['Text']].to_csv(export_prefix+'_text.csv', header=False, index=False)
    print ('Target distribution in the testing data is as follows')
    print ('\n',df_test['Label'].value_counts())


base_dir_path=base_path+'/'+base_folder
change_base_dir(base_dir_path)
download_data(data_url)
extract_tweets_and_labels('training.1600000.processed.noemoticon.csv')

creating directory /content/data
Changing base directory to  /content/data
Step 1: Downloading data
Step 2: Extracting data
Step 3: Reading the data as a dataframe
Read 1600000 lines
Discarding neutral tweets
No of lines in the data after filtering neutral tweets: 1600000
Step 4: Shuffling the data
         Label  ...                                               Text
1502276      4  ...  Andy is adorable! I'm still trying to upload p...
942199       4  ...  @dribblebuster spoke to the shop in teddington...
1099884      4  ...  @Fiona_of_Toorak Bit of a love-hate relationsh...
167952       0  ...  Had a fantastic day with an amazing girl! Just...
1102205      4  ...  @docbaty B/W prepping to be a daddy, sitting o...

[5 rows x 6 columns]
Step 5: Dividing into test and train datasets
Step 6: Exporting the train and test datasets
Exporting training data of rows 1280000
Target distribution in the training data is as follows

 4    640198
0    639802
Name: Label, dtype: int64
Exporting tra

In [0]:
from keras import backend as K
import os



In [7]:
pip install num2words


     |████████████████████████████████| 102kB 2.4MB/s 


In [9]:


import pandas as pd
import numpy as np
from timeit import default_timer as timer
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Embedding
from keras.layers.pooling import GlobalMaxPooling1D,MaxPooling1D
from keras.layers.convolutional import Convolution1D
from keras.layers.core import Lambda
from keras import optimizers
from keras.models import Model
from keras.regularizers import l1
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from IPython.display import SVG
import pydot
from keras.utils.vis_utils import model_to_dot
import re
import io
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
import num2words

random_seed=1
np.random.seed(random_seed)



data_dir = r'/content/data'
embedding_folder = 'vectors'
model_identifier = 'SSWE_Basic_Keras_w_CNTK'

if not os.path.exists(embedding_folder):
    os.makedirs(embedding_folder)

max_sequence_length = 15 # each sentence of the input should be padded to have at least this many tokens
embedding_dim 		= 50 # Embedding layer size
no_filters			= 15 # No of filters for the convolution layer
filter_size			= 5  # Filter size for the convolution layer
trainable 			= True # flag specifying whether the embedding layer weights should be changed during the training or not
batch_size 			= 1024*6 # batch size can be increased to have better gpu utilization
#batch_size 			= 64 # batch size can be increased to have better gpu utilization
no_epochs 			= 5 # No of training epochs



pos_emoticons=["(^.^)","(^-^)","(^_^)","(^_~)","(^3^)","(^o^)","(~_^)","*)",":)",":*",":-*",":]",":^)",":}",
               ":>",":3",":b",":-b",":c)",":D",":-D",":O",":-O",":o)",":p",":-p",":P",":-P",":Þ",":-Þ",":X",
               ":-X",";)",";-)",";]",";D","^)","^.~","_)m"," ~.^","<=8","<3","<333","=)","=///=","=]","=^_^=",
               "=<_<=","=>.<="," =>.>="," =3","=D","=p","0-0","0w0","8D","8O","B)","C:","d'-'","d(>w<)b",":-)",
               "d^_^b","qB-)","X3","xD","XD","XP","ʘ‿ʘ","❤","💜","💚","💕","💙","💛","💓","💝","💖","💞",
               "💘","💗","😗","😘","😙","😚","😻","😀","😁","😃","☺","😄","😆","😇","😉","😊","😋","😍",
               "😎","😏","😛","😜","😝","😮","😸","😹","😺","😻","😼","👍"]

neg_emoticons=["--!--","(,_,)","(-.-)","(._.)","(;.;)9","(>.<)","(>_<)","(>_>)","(¬_¬)","(X_X)",":&",":(",":'(",
               ":-(",":-/",":-@[1]",":[",":\\",":{",":<",":-9",":c",":S",";(",";*(",";_;","^>_>^","^o)","_|_",
               "`_´","</3","<=3","=/","=\\",">:(",">:-(","💔","☹️","😌","😒","😓","😔","😕","😖","😞","😟",
               "😠","😡","😢","😣","😤","😥","😦","😧","😨","😩","😪","😫","😬","😭","😯","😰","😱","😲",
               "😳","😴","😷","😾","😿","🙀","💀","👎"]

# Emails
emailsRegex=re.compile(r'[\w\.-]+@[\w\.-]+')

# Mentions
userMentionsRegex=re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)')

#Urls
urlsRegex=re.compile('r(f|ht)(tp)(s?)(://)(.*)[.|/][^ ]+') # It may not be handling all the cases like t.co without http

#Numerics
numsRegex=re.compile(r"\b\d+\b")

punctuationNotEmoticonsRegex=re.compile(r'(?<=\w)[^\s\w](?![^\s\w])')

emoticonsDict = {}
for i,each in enumerate(pos_emoticons):
    emoticonsDict[each]=' POS_EMOTICON_'+num2words.num2words(i).upper()+' '
    
for i,each in enumerate(neg_emoticons):
    emoticonsDict[each]=' NEG_EMOTICON_'+num2words.num2words(i).upper()+' '
    
# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in emoticonsDict.items())
emoticonsPattern = re.compile("|".join(rep.keys()))


def read_data(filename):   
    """Read the raw tweet data from a file. Replace Emails etc with special tokens"""
    with open(filename, 'r') as f:
    
        all_lines=f.readlines()
        padded_lines=[]
        for line in all_lines:
                    line = emoticonsPattern.sub(lambda m: rep[re.escape(m.group(0))], line.lower().strip())
                    line = userMentionsRegex.sub(' USER ', line )
                    line = emailsRegex.sub(' EMAIL ', line )
                    line=urlsRegex.sub(' URL ', line)
                    line=numsRegex.sub(' NUM ',line)
                    line=punctuationNotEmoticonsRegex.sub(' PUN ',line)
                    line=re.sub(r'(.)\1{2,}', r'\1\1',line)
                    words_tokens=[token for token in TweetTokenizer().tokenize(line)]                   
                    line= ' '.join(token for token in words_tokens )        
                    padded_lines.append(line)
        return padded_lines
    
    
def read_labels(filename):
    """ read the tweet labels from the file"""
    arr= np.genfromtxt(filename, delimiter='\n')
    arr[arr==4]=1 # Encode the positive category as 1
    return arr

# Loading Training and Validation Data
texts 				= []
labels 				= []
nb_train_samples	= 0
nb_valid_samples 	= 0

print ('Loading Training Labels')
train_labels=read_labels(data_dir+'//training_label.csv')

print ('Loading Training data')
train_texts=read_data(data_dir+'//training_text.csv')

print (len(train_labels), len(train_texts))
print ("Using Keras tokenizer to tokenize and build word index")
tokenizer = Tokenizer(lower=False, filters='\n\t?"!') 
train_texts=[each for each in train_texts]
tokenizer.fit_on_texts(train_texts)
sorted_voc = [wc[0] for wc in sorted(tokenizer.word_counts.items(),reverse=True, key= lambda x:x[1]) ]
tokenizer.word_index = dict(list(zip(sorted_voc, list(range(2, len(sorted_voc) + 2)))))
tokenizer.word_index['<PAD>']=0
tokenizer.word_index['<UNK>']=1
word_index = tokenizer.word_index
reverse_dictionary={v:k for (k,v) in tokenizer.word_index.items()}
vocab_size=len(tokenizer.word_index.keys())

print ('Size of the vocab is', vocab_size)



print ('Padding sentences and shuffling the data')
sequences = tokenizer.texts_to_sequences(train_texts)

#Pad the sentences to have consistent length
data = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
labels = to_categorical(np.asarray(train_labels))
indices = np.arange(len(labels))
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

train_x, valid_x, train_y, valid_y=train_test_split(data, labels, test_size=0.2, random_state=random_seed)
train_x=np.array(train_x).astype('float32')
valid_x=np.array(valid_x).astype('float32')
train_y=np.array(train_y)
valid_y=np.array(valid_y)
embedding_matrix = np.zeros((len(word_index) , embedding_dim))
training_word_index=tokenizer.word_index.copy()


print ('Initializing the model')
mcp = ModelCheckpoint('./model_chkpoint', monitor="val_acc", save_best_only=True, save_weights_only=False)

#Creating network
model = Sequential()
model.add(Embedding(len(word_index)+2,
                            embedding_dim,
                            input_length=max_sequence_length,
                            trainable=trainable, name='embedding'))
model.add(Convolution1D(no_filters, filter_size, activation='relu'))
model.add(MaxPooling1D(max_sequence_length - filter_size))
model.add(Flatten())
model.add(Dense(no_filters, activation='tanh'))
model.add(Dense(len(labels[0]), activation='softmax'))

optim=optimizers.Adam(lr=0.1, )
model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['acc'])
model.summary()

start=timer()
hist=model.fit(train_x, train_y,nb_epoch=no_epochs, batch_size=batch_size,validation_data=(valid_x, valid_y),callbacks=[mcp])
end=timer()

# Exporting the Embedding Matrix and Vocabulary
def export_embeddings(model_orig):
    """ export embeddings to file"""
    embedding_weights=pd.DataFrame(model_orig.layers[0].get_weights()[0]).reset_index()
    word_indices_df=pd.DataFrame.from_dict(training_word_index,orient='index').reset_index()
    word_indices_df.columns=['word','index']
    print (word_indices_df.shape,embedding_weights.shape)
    merged=pd.merge(word_indices_df,embedding_weights)
    print (merged.shape)
    merged=merged[[each for each in merged.columns if each!='index']]    
    merged.to_csv(embedding_folder+'//embeddings_{}.tsv'.format(model_identifier), sep='\t', 
              index=False, header=False,float_format='%.6f',encoding='utf-8')
    return embedding_weights, word_indices_df, merged

embedding_weights, word_indices_df, merged_df=export_embeddings(model)

Loading Training Labels
Loading Training data
1280000 1280000
Using Keras tokenizer to tokenize and build word index
Size of the vocab is 304268
Padding sentences and shuffling the data
Initializing the model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 50)            15213500  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11, 15)            3765      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 15)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 15)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                240       
___________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:190: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1024000 samples, validate on 256000 samples
Epoch 1/5
1024000/1024000 [==============================] - 50s 49us/step - loss: 0.5045 - acc: 0.7554 - val_loss: 0.4763 - val_acc: 0.7753
Epoch 2/5
1024000/1024000 [==============================] - 50s 49us/step - loss: 0.4693 - acc: 0.7783 - val_loss: 0.4957 - val_acc: 0.7675
Epoch 3/5
1024000/1024000 [==============================] - 50s 49us/step - loss: 0.4796 - acc: 0.7706 - val_loss: 0.5000 - val_acc: 0.7594
Epoch 4/5
1024000/1024000 [==============================] - 50s 48us/step - loss: 0.5021 - acc: 0.7596 - val_loss: 0.5337 - val_acc: 0.7408
Epoch 5/5
1024000/1024000 [==============================] - 52s 51us/step - loss: 0.5277 - acc: 0.7427 - val_loss: 0.5556 - val_acc: 0.6829
(304268, 2) (304270, 51)
(304268, 52)


In [10]:
import numpy as np
import pandas as pd
import os
import io

random_seed=1

np.random.seed(random_seed)

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, merge
from keras.layers.core import Lambda
from keras import optimizers
from keras import regularizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Embedding , Activation
from nltk.tokenize import TweetTokenizer
import re
import num2words
from timeit import default_timer as timer
from sklearn import  metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:

# Path of the training file'
data_dir = r'/content/data'

# Path of the word vectors
vectors_file = r'/content/data/vectors/embeddings_SSWE_Basic_Keras_w_CNTK.tsv' 

model_identifier='evaluation_SSWE_logistic'
models_dir='model'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

In [0]:

pos_emoticons=["(^.^)","(^-^)","(^_^)","(^_~)","(^3^)","(^o^)","(~_^)","*)",":)",":*",":-*",":]",":^)",":}",
               ":>",":3",":b",":-b",":c)",":D",":-D",":O",":-O",":o)",":p",":-p",":P",":-P",":Þ",":-Þ",":X",
               ":-X",";)",";-)",";]",";D","^)","^.~","_)m"," ~.^","<=8","<3","<333","=)","=///=","=]","=^_^=",
               "=<_<=","=>.<="," =>.>="," =3","=D","=p","0-0","0w0","8D","8O","B)","C:","d'-'","d(>w<)b",":-)",
               "d^_^b","qB-)","X3","xD","XD","XP","ʘ‿ʘ","❤","💜","💚","💕","💙","💛","💓","💝","💖","💞",
               "💘","💗","😗","😘","😙","😚","😻","😀","😁","😃","☺","😄","😆","😇","😉","😊","😋","😍",
               "😎","😏","😛","😜","😝","😮","😸","😹","😺","😻","😼","👍"]

neg_emoticons=["--!--","(,_,)","(-.-)","(._.)","(;.;)9","(>.<)","(>_<)","(>_>)","(¬_¬)","(X_X)",":&",":(",":'(",
               ":-(",":-/",":-@[1]",":[",":\\",":{",":<",":-9",":c",":S",";(",";*(",";_;","^>_>^","^o)","_|_",
               "`_´","</3","<=3","=/","=\\",">:(",">:-(","💔","☹️","😌","😒","😓","😔","😕","😖","😞","😟",
               "😠","😡","😢","😣","😤","😥","😦","😧","😨","😩","😪","😫","😬","😭","😯","😰","😱","😲",
               "😳","😴","😷","😾","😿","🙀","💀","👎"]

# Emails
emailsRegex=re.compile(r'[\w\.-]+@[\w\.-]+')

# Mentions
userMentionsRegex=re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)')

#Urls
urlsRegex=re.compile('r(f|ht)(tp)(s?)(://)(.*)[.|/][^ ]+') # It may not be handling all the cases like t.co without http

#Numerics
numsRegex=re.compile(r"\b\d+\b")

punctuationNotEmoticonsRegex=re.compile(r'(?<=\w)[^\s\w](?![^\s\w])')

emoticonsDict = {}
for i,each in enumerate(pos_emoticons):
    emoticonsDict[each]=' POS_EMOTICON_'+num2words.num2words(i).upper()+' '
    
for i,each in enumerate(neg_emoticons):
    emoticonsDict[each]=' NEG_EMOTICON_'+num2words.num2words(i).upper()+' '
    
# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in emoticonsDict.items())
emoticonsPattern = re.compile("|".join(rep.keys()))

In [0]:
def read_data(filename):
    """Read the raw tweet data from a file. Replace Emails etc with special tokens"""
    with open(filename, 'r') as f:
    
        all_lines=f.readlines()
        padded_lines=[]
        for line in all_lines:
                    line = emoticonsPattern.sub(lambda m: rep[re.escape(m.group(0))], line.lower().strip())
                    line = userMentionsRegex.sub(' USER ', line )
                    line = emailsRegex.sub(' EMAIL ', line )
                    line=urlsRegex.sub(' URL ', line)
                    line=numsRegex.sub(' NUM ',line)
                    line=punctuationNotEmoticonsRegex.sub(' PUN ',line)
                    line=re.sub(r'(.)\1{2,}', r'\1\1',line)
                    words_tokens=[token for token in TweetTokenizer().tokenize(line)]                   
                    line= ' '.join(token for token in words_tokens )        
                    padded_lines.append(line)
        return padded_lines
    
def read_labels(filename):
    """ read the tweet labels from the file"""
    arr= np.genfromtxt(filename, delimiter='\n')
    arr[arr==4]=1 # Encode the positive category as 1
    return arr

# Convert Word Vectors to Sentence Vectors

def load_word_embedding(vectors_file):
    """ Load the word vectors"""
    vectors= np.genfromtxt(vectors_file, delimiter='\t', comments='#--#',dtype=None,
                           names=['Word']+['EV{}'.format(i) for i in range(1,51)])
    # comments have to be changed as some of the tokens are having # in them and then we dont need comments
    vectors_dc={}
    for x in vectors:
        vectors_dc[x['Word'].decode('utf-8','ignore')]=[float(x[each]) for each in ['EV{}'.format(i) for i in range(1,51)]]
    return vectors_dc

def get_sentence_embedding(text_data, vectors_dc):
    sentence_vectors=[]
    
    for sen in text_data:
        tokens=sen.split(' ')
        current_vector=np.array([vectors_dc[tokens[0]] if tokens[0] in vectors_dc else vectors_dc['<UNK>']])
        for word in tokens[1:]:
            if word in vectors_dc:
                current_vector=np.vstack([current_vector,vectors_dc[word]])
            else:
                current_vector=np.vstack([current_vector,vectors_dc['<UNK>']])
        min_max_mean=np.hstack([current_vector.min(axis=0),current_vector.max(axis=0),current_vector.mean(axis=0)])
        sentence_vectors.append(min_max_mean)
    return sentence_vectors


# Model Training

batch_size = 1028*6 # Batch Size should be changed according to the system specifications to have better utilization of GPU
nb_epoch = 10

def init_model():
    output_dim = no_classes = len(to_categorical(train_y)[0])
    input_dim=150
    model = Sequential() 
    model.add(Dense(output_dim, input_dim=input_dim, activation='softmax',activity_regularizer=regularizers.l1_l2(1))) 
    return model

def cv_estimate(n_splits,X_train, y_train):
    best_score, best_model= None,None
    cv = KFold(n_splits=n_splits)
    
    i=0
    for train, test in cv.split(X_train, y_train):
        model=init_model()
        mcp = ModelCheckpoint('./model_chkpoint_{}'.format(i), monitor="val_acc",
                      save_best_only=True, save_weights_only=False)
        model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

        model.fit(X_train[train], to_categorical(y_train[train]),epochs=nb_epoch,batch_size=batch_size, callbacks=[mcp],
                  validation_split=0.2)
        current_score= model.evaluate(X_train[test], to_categorical(y_train[test]))[0] # Getting the loss
        print ('\n Fold {} Current score {}'.format(i+1, current_score))
                
        if i==0:
            best_score=current_score
            best_model=model
        else:
            print (current_score)
            if current_score<best_score:
                best_score=current_score
                best_model=model
        i+=1

    return  best_model

In [0]:
print ('Step 1: Loading Training data')
train_texts=read_data(data_dir+'/training_text.csv')
print ('Reading training labels')
train_labels=read_labels(data_dir+'/training_label.csv')


In [19]:

print ("Step 2: Load Word Vectors")
vectors_dc=load_word_embedding(vectors_file)
len(vectors_dc)

print ("Step 3: Converting the word vectors to sentence vectors")
train_sentence_vectors=get_sentence_embedding(train_texts,vectors_dc)
print (len(train_sentence_vectors), len(train_labels), len(train_texts))
print (" Encoding the input")
train_x=train_sentence_vectors
train_y=train_labels
train_x=np.array(train_x).astype('float32')
train_y=np.array(train_y)

print ('Step 4: Logistic regression model using Keras')
best_model=cv_estimate(3,train_x, train_y)

print ("Step 5: Saving the model")
best_model.save(models_dir+'//'+model_identifier)

Step 2: Load Word Vectors


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.


Step 3: Converting the word vectors to sentence vectors
1280000 1280000 1280000
 Encoding the input
Step 4: Logistic regression model using Keras
Train on 682666 samples, validate on 170667 samples
Epoch 1/10
682666/682666 [==============================] - 1s 2us/step - loss: 6218.9968 - accuracy: 0.5622 - val_loss: 6174.2761 - val_accuracy: 0.5388
Epoch 2/10
104856/682666 [===>..........................] - ETA: 1s - loss: 6233.2565 - accuracy: 0.5629

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


682666/682666 [==============================] - 1s 2us/step - loss: 6220.5022 - accuracy: 0.5644 - val_loss: 6176.7012 - val_accuracy: 0.6383
Epoch 3/10
682666/682666 [==============================] - 1s 2us/step - loss: 6218.8402 - accuracy: 0.5737 - val_loss: 6175.7766 - val_accuracy: 0.6556
Epoch 4/10
682666/682666 [==============================] - 1s 2us/step - loss: 6219.1211 - accuracy: 0.5687 - val_loss: 6173.3127 - val_accuracy: 0.6631
Epoch 5/10
682666/682666 [==============================] - 1s 2us/step - loss: 6220.0692 - accuracy: 0.5683 - val_loss: 6177.1511 - val_accuracy: 0.5311
Epoch 6/10
682666/682666 [==============================] - 1s 2us/step - loss: 6217.8137 - accuracy: 0.5840 - val_loss: 6172.1788 - val_accuracy: 0.6094
Epoch 7/10
682666/682666 [==============================] - 1s 2us/step - loss: 6220.7768 - accuracy: 0.5651 - val_loss: 6177.7819 - val_accuracy: 0.5554
Epoch 8/10
682666/682666 [==============================] - 1s 2us/step - loss: 6216.31